Validador de descredenciamento para estabelecimento com comportamento fraudulento:

O objetivo desse codigo é fazer uma analise em cime de um pequeno banco de dados com as caracteristicas de:

ID da Transação,
Data da compra,
Horario, Valor,
Razão do Chargeback,
ID Estabelecimento,
ID Portador e
Idade do Estabelecimento

afim de buscar caracteristicas autofradulentas evitando um prejuizo por parte do adquirente com futuros prejuizos

--------------------------------------------------------------------------------
Pré-processamento de Datas e Horários

Importação do dataset: O arquivo dataset_anti_chargeback.xlsx é carregado para análise.

Tratamento da Data: A coluna Data é convertida para o tipo datetime, facilitando operações de tempo.

Criação de Ano/Mês (AnoMes): A partir da data, é criada uma coluna no formato YYYY-MM, permitindo análises agregadas por mês.

Tratamento do Horário: A coluna Horário é convertida para formato HH:MM. Valores inválidos são tratados como nulos.

Extração de dia da semana e mês: Novas colunas (Dia_Semana e Mes) ajudam na análise de padrões temporais.

Classificação do Período do Dia: Criada a feature Periodo_Dia, que categoriza as transações em Manhã, Tarde, Noite ou Madrugada.

In [ ]:
import pandas as pd

dados = pd.read_excel(r'/content/dataset_anti_chargeback.xlsx')
dados["Data"] = pd.to_datetime(dados["Data"], format="%d/%m/%Y")


dados["AnoMes"] = dados["Data"].dt.to_period("M")

dados['Horário'] = pd.to_datetime(dados['Horário'], errors='coerce').dt.strftime('%H:%M')

dados['Dia_Semana'] = dados['Data'].dt.day_name()
dados['Mes'] = dados['Data'].dt.month

def periodo_dia(hora):
    hora_int = int(hora.split(':')[0])
    if 6 <= hora_int < 12: return 'Manhã'
    elif 12 <= hora_int < 18: return 'Tarde'
    elif 18 <= hora_int < 24: return 'Noite'
    else: return 'Madrugada'

dados['Periodo_Dia'] = dados['Horário'].apply(periodo_dia)


/tmp/ipython-input-2407002865.py:10: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dados['Horário'] = pd.to_datetime(dados['Horário'], errors='coerce').dt.strftime('%H:%M')


In [ ]:
import numpy as np
dados['recebeu_chargeback'] = np.where(dados['Motivo do Chargeback'].notna(), 1,0)

Criação de Métricas de Chargeback

Neste trecho, são criadas diversas variáveis derivadas que ajudam a identificar padrões de comportamento dos estabelecimentos e portadores:

In [ ]:

dados['QTD_CHARGEBACK_GERAL_ESTAB'] = dados.groupby('ID Estabelecimento')['recebeu_chargeback'].transform('sum')
dados['QTD_CHARGEBACK_MES'] = dados.groupby(['ID Estabelecimento', 'AnoMes'])['recebeu_chargeback'].transform('sum')
dados['QTD_CHARGEBACK_PORTADOR'] = dados.groupby('ID Portador')['recebeu_chargeback'].transform('sum')

dados['QTD_COMPRA_GERAL'] = dados.groupby('ID Estabelecimento')['ID Transação'].transform('count')
dados['QTD_COMPRA_MES'] = dados.groupby(['ID Estabelecimento', 'AnoMes'])['ID Transação'].transform('count')

dados['TAXA_CHARGEBACK_GERAL_ESTAB'] = dados['QTD_CHARGEBACK_GERAL_ESTAB'] / dados['QTD_COMPRA_GERAL']
dados['TAXA_CHARGEBACK_MES'] = dados['QTD_CHARGEBACK_MES'] / dados['QTD_COMPRA_MES']

dados['VLR_GERAL_ESTAB'] = dados.groupby('ID Estabelecimento')['Valor (R$)'].transform('sum')
dados['VLR_MES'] = dados.groupby(['ID Estabelecimento', 'AnoMes'])['Valor (R$)'].transform('sum')

dados['VALOR_CHARGEBACK_ESTAB'] = dados.groupby('ID Estabelecimento')['Valor (R$)'].transform(
    lambda x: x[dados.loc[x.index, 'recebeu_chargeback'] == 1].sum()
)

dados['VALOR_SEM_CHARGEBACK_ESTAB'] = dados.groupby('ID Estabelecimento')['Valor (R$)'].transform(
    lambda x: x[dados.loc[x.index, 'recebeu_chargeback'] == 0].sum()
)


dados['TAXA_MEDIA_CHARGEBACK_ESTAB'] = dados['VALOR_CHARGEBACK_ESTAB'] / dados['VALOR_SEM_CHARGEBACK_ESTAB']
dados['PERC_VALOR_CHARGEBACK_ESTAB'] = dados['VALOR_CHARGEBACK_ESTAB'] / (
    dados['VALOR_CHARGEBACK_ESTAB'] + dados['VALOR_SEM_CHARGEBACK_ESTAB']
)


Análise de Comportamento do Portador

Além da visão do estabelecimento, também é importante entender o padrão de compras do portador (cliente).

Frequência mensal de compras

Frequencia_Portador: conta quantas compras cada portador realizou em cada mês.

Ajuda a identificar reincidência em portadores que concentram muitas transações.

Construção de data e hora completa da transação

DataHora: junção da data (Data) com o horário (Horário) da compra, convertida para o formato datetime.

Permite ordenar e calcular intervalos entre compras.

Intervalo entre compras

Intervalo_Entre_Compras_Dias: mede o tempo (em dias) entre uma compra e a seguinte do mesmo portador.

Portadores com intervalos muito curtos podem indicar comportamento suspeito (ex.: fraude em massa em pouco tempo).

In [ ]:

dados['Frequencia_Portador'] = dados.groupby(['ID Portador', 'AnoMes'])['ID Transação'].transform('count')

dados['DataHora'] = pd.to_datetime(dados['Data'].astype(str) + ' ' + dados['Horário'].astype(str), errors='coerce')
dados = dados.sort_values(['ID Portador', 'DataHora'])
dados['Intervalo_Entre_Compras_Dias'] = dados.groupby('ID Portador')['DataHora'].diff().dt.total_seconds().fillna(0) / (3600*24)


Análise da Idade do Estabelecimento

O tempo de existência de um estabelecimento é um fator crítico para análise antifraude.

Esse passo é essencial porque fraudes são muito mais comuns em estabelecimentos novos, criados com o único objetivo de gerar prejuízo rapidamente.

In [ ]:

dados['Idade ESTAB'] = pd.to_datetime(dados['Idade ESTAB'])
dados['IDADE_ESTAB_DIAS'] = (dados['Data'] - dados['Idade ESTAB']).dt.days


dados['FAIXA_IDADE_ESTAB'] = pd.cut(
    dados['IDADE_ESTAB_DIAS'],
    bins=[0, 7, 30, 90, 365, float('inf')],
    labels=['0-7 dias', '8-30 dias', '1-3 meses', '3-12 meses', '+1 ano']
)


def calcular_score_com_idade(row):
    base_score = row['TAXA_MEDIA_CHARGEBACK_ESTAB']
    idade_dias = row['IDADE_ESTAB_DIAS']
    if idade_dias <= 7:
        return base_score * 1.5
    elif idade_dias <= 30:
        return base_score * 1.3
    elif idade_dias <= 90:
        return base_score * 1.1
    else:
        return base_score * 0.9

dados['SCORE_RISCO_COM_IDADE'] = dados.apply(calcular_score_com_idade, axis=1)

dados['ESTAB_NOVO'] = (dados['IDADE_ESTAB_DIAS'] <= 30).astype(int)
dados['ESTAB_MUITO_NOVO'] = (dados['IDADE_ESTAB_DIAS'] <= 7).astype(int)


Após os cálculos de transações, chargebacks, valores e frequência dos portadores, é gerado um resumo consolidado por estabelecimento.

Cálculo da frequência média de chargebacks por portador

Freq_CB_Media: média de quantas transações o mesmo portador realizou antes de gerar um chargeback.

Se a frequência é baixa, o risco é maior (ex.: portadores diferentes fazendo chargebacks rapidamente).

Resumo consolidado por estabelecimento (resumo_estab)
Inclui:

Quantidade total de chargebacks.

Taxa geral de chargeback.

Percentual de valor perdido em chargeback.

Valores com e sem chargeback.

Quantidade total de compras.

Frequência média de chargeback por portador.

Regras de decisão para ação sobre o estabelecimento
A função acao_estab_com_frequencia aplica as regras:

🔴 DESCREDENCIAR →

Taxa de chargeback > 20% ou

Valor de chargeback > 25% do total transacionado

E frequência média de chargeback por portador < 5.

⚠️ ALERTA →

Taxa de chargeback entre 5% e 20% ou

Valor de chargeback entre 10% e 25%.

✅ OK → risco dentro dos limites aceitáveis.

Divisão final dos estabelecimentos

descredenciar: estabelecimentos de alto risco que devem ser imediatamente bloqueados/descredenciados.

alerta: estabelecimentos com risco moderado, que precisam de monitoramento mais próximo.

In [ ]:

freq_cb = dados[dados['recebeu_chargeback'] == 1].groupby('ID Estabelecimento')['Frequencia_Portador'].mean().reset_index()
freq_cb.rename(columns={'Frequencia_Portador': 'Freq_CB_Media'}, inplace=True)

resumo_estab = dados.groupby('ID Estabelecimento').agg({
    'QTD_CHARGEBACK_GERAL_ESTAB': 'first',
    'TAXA_CHARGEBACK_GERAL_ESTAB': 'first',
    'PERC_VALOR_CHARGEBACK_ESTAB': 'first',
    'VALOR_CHARGEBACK_ESTAB': 'first',
    'VALOR_SEM_CHARGEBACK_ESTAB': 'first',
    'QTD_COMPRA_GERAL': 'first'
}).reset_index()


resumo_estab = resumo_estab.merge(freq_cb, on='ID Estabelecimento', how='left')
resumo_estab['Freq_CB_Media'] = resumo_estab['Freq_CB_Media'].fillna(0)


def acao_estab_com_frequencia(row):
    freq = row['Freq_CB_Media']
    if (row['TAXA_CHARGEBACK_GERAL_ESTAB'] > 0.2 or row['PERC_VALOR_CHARGEBACK_ESTAB'] > 0.25) and freq < 5:
        return 'DESCREDENCIAR'
    elif (0.05 < row['TAXA_CHARGEBACK_GERAL_ESTAB'] <= 0.2 or 0.1 < row['PERC_VALOR_CHARGEBACK_ESTAB'] <= 0.25):
        return 'ALERTA'
    else:
        return 'OK'

resumo_estab['AÇÃO_ESTAB'] = resumo_estab.apply(acao_estab_com_frequencia, axis=1)


descredenciar = resumo_estab[resumo_estab['AÇÃO_ESTAB'] == 'DESCREDENCIAR']
alerta = resumo_estab[resumo_estab['AÇÃO_ESTAB'] == 'ALERTA']

print("Descredenciar:")
print(descredenciar.head())

print("\nAlertas:")
print(alerta.head())


Descredenciar:
   ID Estabelecimento  QTD_CHARGEBACK_GERAL_ESTAB  \
6        estab_130674                           8   
8        estab_138213                           8   
17       estab_162094                          10   
34       estab_212017                          23   
49       estab_291076                          16   

    TAXA_CHARGEBACK_GERAL_ESTAB  PERC_VALOR_CHARGEBACK_ESTAB  \
6                      0.258065                     0.248014   
8                      0.216216                     0.233698   
17                     0.322581                     0.292146   
34                     0.676471                     0.741812   
49                     0.516129                     0.566669   

    VALOR_CHARGEBACK_ESTAB  VALOR_SEM_CHARGEBACK_ESTAB  QTD_COMPRA_GERAL  \
6                 16758.34                    50811.67                31   
8                 20115.44                    65959.00                37   
17                23356.29                    56590.9

In [ ]:
dados.columns

Index(['ID Transação', 'Data', 'Horário', 'Valor (R$)', 'Motivo do Chargeback',
       'ID Estabelecimento', 'ID Portador', 'Idade ESTAB', 'AnoMes',
       'Dia_Semana', 'Mes', 'Periodo_Dia', 'recebeu_chargeback',
       'QTD_CHARGEBACK_GERAL_ESTAB', 'QTD_CHARGEBACK_MES',
       'QTD_CHARGEBACK_PORTADOR', 'QTD_COMPRA_GERAL', 'QTD_COMPRA_MES',
       'TAXA_CHARGEBACK_GERAL_ESTAB', 'TAXA_CHARGEBACK_MES', 'VLR_GERAL_ESTAB',
       'VLR_MES', 'VALOR_CHARGEBACK_ESTAB', 'VALOR_SEM_CHARGEBACK_ESTAB',
       'TAXA_MEDIA_CHARGEBACK_ESTAB', 'PERC_VALOR_CHARGEBACK_ESTAB',
       'Frequencia_Portador', 'DataHora', 'Intervalo_Entre_Compras_Dias',
       'IDADE_ESTAB_DIAS', 'FAIXA_IDADE_ESTAB', 'SCORE_RISCO_COM_IDADE',
       'ESTAB_NOVO', 'ESTAB_MUITO_NOVO'],
      dtype='object')

Criação de mensagem utilizada para gerar o alerta via whatsapp

In [ ]:

descredenciar = resumo_estab[resumo_estab['AÇÃO_ESTAB'] == 'DESCREDENCIAR']
alerta = resumo_estab[resumo_estab['AÇÃO_ESTAB'] == 'ALERTA']

def formatar_mensagem(df, tipo='DESCREDENCIAR'):
    mensagens = []
    for _, row in df.iterrows():
        if tipo == 'DESCREDENCIAR':
            simbolo = "🔴"
            acao = 'Descredenciado'
            acao_txt = 'Ação Realizada'
        else:
            simbolo = "⚠️"
            acao = 'ANALISAR'
            acao_txt = 'Ação Recomendada'

        msg = f"""{simbolo} ALERTA CHARGEBACK
Estabelecimento: {row['ID Estabelecimento']}
Taxa Chargeback: {row['TAXA_CHARGEBACK_GERAL_ESTAB']*100:.0f}%
Valor Chargeback: R$ {row['VALOR_CHARGEBACK_ESTAB']:,.2f}
Valor Transacional: R$ {row['VALOR_SEM_CHARGEBACK_ESTAB']:,.2f}

{acao_txt}: {acao}
-------------------------"""
        mensagens.append(msg)
    return mensagens


mensagens_descredenciar = formatar_mensagem(descredenciar, tipo='DESCREDENCIAR')
mensagens_alerta = formatar_mensagem(alerta, tipo='ALERTA')

for msg in mensagens_descredenciar[:3]:
    print(msg)

for msg in mensagens_alerta[:3]:
    print(msg)


🔴 ALERTA CHARGEBACK
Estabelecimento: estab_130674
Taxa Chargeback: 26%
Valor Chargeback: R$ 16,758.34
Valor Transacional: R$ 50,811.67

Ação Realizada: Descredenciado
-------------------------
🔴 ALERTA CHARGEBACK
Estabelecimento: estab_138213
Taxa Chargeback: 22%
Valor Chargeback: R$ 20,115.44
Valor Transacional: R$ 65,959.00

Ação Realizada: Descredenciado
-------------------------
🔴 ALERTA CHARGEBACK
Estabelecimento: estab_162094
Taxa Chargeback: 32%
Valor Chargeback: R$ 23,356.29
Valor Transacional: R$ 56,590.91

Ação Realizada: Descredenciado
-------------------------
⚠️ ALERTA CHARGEBACK
Estabelecimento: estab_117451
Taxa Chargeback: 13%
Valor Chargeback: R$ 18,002.30
Valor Transacional: R$ 104,185.55

Ação Recomendada: ANALISAR
-------------------------
⚠️ ALERTA CHARGEBACK
Estabelecimento: estab_118553
Taxa Chargeback: 9%
Valor Chargeback: R$ 10,625.45
Valor Transacional: R$ 115,973.49

Ação Recomendada: ANALISAR
-------------------------
⚠️ ALERTA CHARGEBACK
Estabelecimento: es

In [ ]:
!pip install twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.9 MB/s eta 0:00:00


In [ ]:
from twilio.rest import Client


input da mensagem no whatsap alertando o descredenciamento de estabelecimento e o alerta para analise para os com risco moderado

In [ ]:
from twilio.rest import Client

account_sid = "xxxxxxx"
auth_token = "xxxxxxx"
client = Client(account_sid, auth_token)

message = client.messages.create(
    from_="whatsapp:+18653918990",
    body=(
      "-------------------------\n"
      "🔴 ALERTA CHARGEBACK\n"
      "Estabelecimento: estab_138213\n"
      "Taxa Chargeback: 22%\n"
      "Valor Chargeback: R$ 20,115.44\n"
      "Valor Transacional: R$ 65,959.00\n"
      "Ação Realizada: Descredenciado\n"
      "-------------------------\n"
      "🔴 ALERTA CHARGEBACK\n"
      "Estabelecimento: estab_162094\n"
      "Taxa Chargeback: 32%\n"
      "Valor Chargeback: R$ 23,356.29\n"
      "Valor Transacional: R$ 56,590.91\n"
      "Ação Realizada: Descredenciado\n"
      "-------------------------\n"
      "⚠️ ALERTA CHARGEBACK\n"
      "Estabelecimento: estab_128527\n"
      "Taxa Chargeback: 17%\n"
      "Valor Chargeback: R$ 12,956.05\n"
      "Valor Transacional: R$ 100,815.32\n\n"
      "Ação Recomendada: ANALISAR\n"
      "-------------------------\n"
      "🔴 ALERTA CHARGEBACK\n"
      "Estabelecimento: estab_138213\n"
      "Taxa Chargeback: 22%\n"
      "Valor Chargeback: R$ 20,115.44\n"
      "Valor Transacional: R$ 65,959.00\n\n"
      "Ação Realizada: Descredenciado\n"
      "-------------------------\n"
      "⚠️ ALERTA CHARGEBACK\n"
      "Estabelecimento: estab_118553\n"
      "Taxa Chargeback: 9%\n"
      "Valor Chargeback: R$ 10,625.45\n"
      "Valor Transacional: R$ 115,973.49\n"

      "Ação Recomendada: ANALISAR\n"
      "-------------------------\n"
      "⚠️ ALERTA CHARGEBACK\n"
      "Estabelecimento: estab_117451\n"
      "Taxa Chargeback: 13%\n"
      "Valor Chargeback: R$ 18,002.30\n"
      "Valor Transacional: R$ 104,185.55\n"
      "Ação Recomendada: ANALISAR\n"
       "-------------------------\n"
    ),
   to="whatsapp:+5511xxxxxxxx"
)

print(message.sid)


In [ ]:
dados

,ID Transação,Data,Horário,Valor (R$),Motivo do Chargeback,ID Estabelecimento,ID Portador,Idade ESTAB,AnoMes,Dia_Semana,...,TAXA_MEDIA_CHARGEBACK_ESTAB,PERC_VALOR_CHARGEBACK_ESTAB,Frequencia_Portador,DataHora,Intervalo_Entre_Compras_Dias,IDADE_ESTAB_DIAS,FAIXA_IDADE_ESTAB,SCORE_RISCO_COM_IDADE,ESTAB_NOVO,ESTAB_MUITO_NOVO
441,trx_503886,2025-05-10,17:14,4930.77,NaN,estab_454685,ch_101729,2025-05-05,2025-05,Saturday,...,0.102446,0.092926,1,2025-05-10 17:14:00,0.000000,5,0-7 dias,0.153669,1,1
1780,trx_916445,2025-06-04,02:09,750.75,NaN,estab_586153,ch_101729,2025-04-26,2025-06,Wednesday,...,0.077767,0.072155,2,2025-06-04 02:09:00,24.371528,39,1-3 meses,0.085543,0,0
2322,trx_591663,2025-06-15,16:51,1190.81,NaN,estab_297973,ch_101729,2025-05-04,2025-06,Sunday,...,0.101428,0.092088,2,2025-06-15 16:51:00,11.612500,42,1-3 meses,0.111571,0,0
3146,trx_856062,2025-07-01,06:41,4627.80,NaN,estab_503177,ch_101729,2025-04-28,2025-07,Tuesday,...,0.113031,0.101553,3,2025-07-01 06:41:00,15.576389,64,1-3 meses,0.124335,0,0
3966,trx_494589,2025-07-17,22:00,4719.24,NaN,estab_212904,ch_101729,2025-05-03,2025-07,Thursday,...,0.133667,0.117907,3,2025-07-17 22:00:00,16.638194,75,1-3 meses,0.147034,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275,trx_386800,2025-10-03,15:29,565.94,Não recebeu o produto,estab_628930,ch_999752,2025-04-28,2025-10,Friday,...,0.081676,0.075509,2,2025-10-03 15:29:00,8.117361,158,3-12 meses,0.073508,0,0
7532,trx_368774,2025-10-10,02:16,3386.11,NaN,estab_303668,ch_999752,2025-04-26,2025-10,Friday,...,0.094355,0.086220,2,2025-10-10 02:16:00,6.449306,167,3-12 meses,0.084920,0,0
8566,trx_148719,2025-11-04,12:05,4402.53,NaN,estab_166581,ch_999752,2025-05-16,2025-11,Tuesday,...,0.000000,0.000000,2,2025-11-04 12:05:00,25.409028,172,3-12 meses,0.000000,0,0
9292,trx_945106,2025-11-22,17:07,4235.53,NaN,estab_166581,ch_999752,2025-05-16,2025-11,Saturday,...,0.000000,0.000000,2,2025-11-22 17:07:00,18.209722,190,3-12 meses,0.000000,0,0
